<a href="https://colab.research.google.com/github/pszemraj/ai-msgbot/blob/main/colab-notebooks/%5BGPT2_dailydialogue%5D_aitextgen_text_generation_%2B_training_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

This notebook is based on the original tutorial from `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).




In [1]:
!nvidia-smi

Thu Oct 21 23:19:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## formatting

In [2]:
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

# setup

In [3]:
# update torch in case using a A100 GPU
!pip3 install torch==1.9.1+cu111 -f -q https://download.pytorch.org/whl/torch_stable.html

!pip3 install cudatoolkit

Looking in links: -q
     |████████████████████████████████| 253 kB 511 kB/s 
  ERROR: Cannot unpack file /tmp/pip-unpack-2ldte1l4/torch_stable.html (downloaded from /tmp/pip-req-build-nxej9t5y, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-nxej9t5y
ERROR: Could not find a version that satisfies the requirement cudatoolkit (from versions: none)
ERROR: No matching distribution found for cudatoolkit


In [4]:
!pip install -q aitextgen

import logging

logging.basicConfig(
    format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

     |████████████████████████████████| 572 kB 16.6 MB/s 
     |████████████████████████████████| 2.9 MB 56.3 MB/s 
     |████████████████████████████████| 87 kB 7.8 MB/s 
     |████████████████████████████████| 925 kB 59.7 MB/s 
     |████████████████████████████████| 829 kB 66.8 MB/s 
     |████████████████████████████████| 596 kB 63.7 MB/s 
     |████████████████████████████████| 282 kB 68.6 MB/s 
     |████████████████████████████████| 125 kB 78.1 MB/s 
     |████████████████████████████████| 1.3 MB 70.7 MB/s 
     |████████████████████████████████| 895 kB 66.8 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 69.6 MB/s 
     |████████████████████████████████| 271 kB 80.3 MB/s 
     |████████████████████████████████| 160 kB 85.4 MB/s 


In [5]:
mount_gdrive()


Mounted at /content/drive


## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [6]:
!nvidia-smi

Thu Oct 21 23:20:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

## 

In [7]:
model_size = "355M" #@param ["355M", "774M"]
load_from_folder = True #@param {type:"boolean"}
load_folder_dir = "/content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_355M" #@param {type:"string"}


In [8]:
if load_from_folder:
    ai = aitextgen(model_folder=load_folder_dir, to_gpu=True,
                   gradient_checkpointing=True)
else:
    ai = aitextgen(tf_gpt2=model_size, to_gpu=True,
                gradient_checkpointing=True)
# ai = aitextgen(tf_gpt2="124M", to_gpu=True)
# https://huggingface.co/distilgpt2
# Comment out the above line and uncomment the below line to use GPT Neo instead.
# model_size
# ai = aitextgen(model='gpt2-medium', 
#                to_gpu=True, 
#                gradient_checkpointing=True)

10/21/2021 23:20:28 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_355M.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
10/21/2021 23:20:44 — INFO — aitextgen — GPT2 loaded with 354M parameters.
10/21/2021 23:20:44 — INFO — aitextgen — Gradient checkpointing enabled for model training.
10/21/2021 23:20:44 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


# load training data


- <font color="orange"> combine any other data from the _DailyDialogue_ dataset and do a final pass of training (with different data) </font>

In [9]:
dl_link = "https://www.dropbox.com/s/zngfhi9zw438fxk/daily_dialogue_augment.txt?dl=1" #@param {type:"string"}


In [10]:
# download test image
from urllib import request
from os.path import join
import os
vm_wd = os.getcwd()
local_name = join(vm_wd, "dailydialogue.txt")
request.urlretrieve(dl_link, local_name)


('/content/dailydialogue.txt', <http.client.HTTPMessage at 0x7fe94d380150>)

In [11]:
# adjust names 


with open(local_name, 'r', encoding='utf-8', errors='ignore') as fi:
    orig_lines = fi.readlines()

from tqdm.auto import tqdm

upd_lines = []

for line in tqdm(orig_lines, total=len(orig_lines), desc="replacing speaker names"):
    fixline = line.replace("speaker a", "john smith")
    fixline = fixline.replace("speaker b", "nancy sellers")
    upd_lines.append(fixline)

local_namev2 = join(vm_wd, "V2dailydialogue.txt")

with open(local_namev2, 'w', encoding='utf-8', errors='ignore') as fo:
    fo.writelines(upd_lines)


replacing speaker names:   0%|          | 0/216768 [00:00<?, ?it/s]

In [12]:
import pprint as pp

pp.pprint(upd_lines[:10])

['john smith:\n',
 'say, jim, how about going for a few beers after dinner?\n',
 '\n',
 'nancy sellers:\n',
 'you know that is tempting but is really not good for our fitness.\n',
 '\n',
 'john smith:\n',
 'what do you mean? it will help us to relax.\n',
 '\n',
 'nancy sellers:\n']


In [13]:
file_name = local_namev2

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

In [14]:
# copy_file_from_gdrive(local_name)

# Train / Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [15]:
import gc, os
from os.path import join

base_dir = "/content/drive/MyDrive/Programming/AI_peter"
temp_gpu_path = join(base_dir, "gp2_dailydialogue_{}".format(model_size))
os.makedirs(temp_gpu_path, exist_ok=True)
gc.collect()

0

In [17]:
# DO NOT USE WARMUP STEPS

ai.train(file_name,
         output_dir=temp_gpu_path,
         line_by_line=False,
         from_cache=False,
         num_steps=75000, # takes about 5 hours on 16 gb GPU for 75000
         generate_every=1500,
         max_grad_norm=0.8,
         save_every=1000,
         gradient_accumulation_steps=1,
         save_gdrive=False,
         learning_rate=1e-3,
        #  fp16=True,
         batch_size=1, 
        #  fp16_opt_level="O1",
         )

10/21/2021 23:25:50 — INFO — aitextgen — Loading text from /content/V2dailydialogue.txt with generation length of 1024.


  0%|          | 0/216768 [00:00<?, ?it/s]

10/21/2021 23:25:50 — INFO — aitextgen.TokenDataset — Encoding 216,768 sets of tokens from /content/V2dailydialogue.txt.
10/21/2021 23:25:54 — WARNING — aitextgen — pytorch_model.bin already exists in //content/drive/MyDrive/Programming/AI_peter/gp2_dailydialogue_355M and will be overwritten!
10/21/2021 23:25:54 — INFO — pytorch_lightning.utilities.distributed — GPU available: True, used: True
10/21/2021 23:25:54 — INFO — pytorch_lightning.utilities.distributed — TPU available: False, using: 0 TPU cores
10/21/2021 23:25:54 — INFO — pytorch_lightning.utilities.distributed — IPU available: False, using: 0 IPUs
10/21/2021 23:25:54 — INFO — pytorch_lightning.accelerators.gpu — LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/75000 [00:00<?, ?it/s]

1,000 steps reached: saving model to //content/drive/MyDrive/Programming/AI_peter/gp2_dailydialogue_355M
1,500 steps reached: generating sample texts.
! that will be a bit too hot for us, isn't it?

john smith:
well, i don't think so. your credit manager called me this book as a father's day gift, but for he, some bottles of champagne.

nancy sellers:
ok, i'm sure it gets to be back to you about the next week.

john smith:
no doubt, but i think i'm going to china and i need some traveler's checks.

nancy sellers:
don't worry about it. our embassy offers passport course and all our employees a traveler's checks.

john smith:
oh, i see. as the manager of my bank, the salary is not paid yet.

nancy sellers:
i'm sorry to say that's not on the scale. i will have to ask my credit manager to come over.

john smith:
hi, mr. purchaseley.

nancy sellers:
hello, i'm gary.

john smith:
hi, this is my friend mary.

nancy sellers:
yes, i am looking for a new way to choose.

john
2,000 steps reached:

10/22/2021 06:48:32 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/MyDrive/Programming/AI_peter/gp2_dailydialogue_355M


In [18]:
save_path = "/content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_{}".format(model_size)

In [19]:
import os
os.makedirs(save_path, exist_ok=True)
ai.save(save_path)


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.


# Use a Train Model for Generation

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [20]:
!nvidia-smi

Fri Oct 22 06:48:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    50W / 300W |   4491MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
mount_gdrive()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# best model thus far @ 1.3B parameters and tuned for 50k steps
# from_folder = "/content/drive/MyDrive/Programming/AI_peter/GPT-Neo-1B-V1"

from_folder = save_path

if len(from_folder) > 2:

    for file in ["pytorch_model.bin", "config.json"]:
        if from_folder:
            copy_file_from_gdrive(file, from_folder)
        else:
            copy_file_from_gdrive(file)

    ai = aitextgen(model_folder=from_folder, to_gpu=True)
else:
    ai = aitextgen(model_folder=".", to_gpu=True)


10/22/2021 06:49:03 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/Programming/AI_peter/gpt2_dailydialogue_gpu_355M.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
10/22/2021 06:49:09 — INFO — aitextgen — GPT2 loaded with 354M parameters.
10/22/2021 06:49:09 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


## Generate Text From The Trained Model


`generate()` without any parameters generates a single text from the loaded model to the console.

In [23]:
ai.generate(n=3, max_length=256, 
            temperature=1.0, top_p=0.9)

. the first thing we do is make sure that our quality is high.

nancy sellers:
do you think it's that way to stop wrinkles our also have to trim the invasion.

john smith:
what! you've got to cut out about this mess! you won't believe what it can do.

nancy sellers:
for instance?

john smith:
it has a built-in camera unit that takes digital color stills. it also plays mp3 files, which i can save on media cards and then transfer to my hard drive of my computer.

nancy sellers:
wow! anything else?

john smith:
this one comes with its own folding keyboard. it has a pre-installed handwriting recognition program and a training program.

nancy sellers:
what training program are you going to have to get?

john smith:
i want to start training.

nancy sellers:
me too.

john smith:
yes, but do you have any experience?

nancy sellers:
yes. i have a tefl certificate and a ticket. can you name it?

john smith:

salary that you succeed in salary.

nancy sellers:
yes, i guess i thought i would just w

In [24]:
ai.generate(prompt="give me a good pickup line!", temperature=1,
            min_length=10, batch_size =20)

give me a good pickup line!

nancy sellers:
okay, let's go!

john smith:
till, you're going to quit!

nancy sellers:
i'll try to quit my job.

john smith:
i can stay longer, because i agree.

nancy sellers:
is my boss going to get off from me?

john smith:
yes, he'll be there.

nancy sellers:
please turn the good news for him.

john smith:
ok! can you tell me he's locked?

nancy sellers:
i'm afraid not. he's no. michael probably forgot to assign a locker to you.

john smith:
no problem. i think i'll let you know.

nancy sellers:
did you use your understanding of responsibility in the trade market?

john smith:
yes, i noticed it was not very complicated. it's a good idea to work as a public banker.

nancy sellers:
we should be able to finish our work and complete our request.

john smith:
yes, i want to


If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

In [25]:
ai.generate(
    n=3, batch_size=25, prompt="i just", max_length=256, 
    temperature=1.0, top_p=0.9
)

i just wanted to let you to know that we had decided to offer the position to someone else.we considered you are a good candidate.

john smith:
you are a great attitude, but vane is not worth it.

nancy sellers:
that's because you are not responsible for anyone starting salary.

john smith:
i don't know what to do.

nancy sellers:
have you thought about what you are going to do now?

john smith:
i'm not going to pay anything.

nancy sellers:
you get a pair of questions. but remember to incorporate a little things up.

john smith:
first, you should check the schedule and see which trains go to suzhou. make sure which train you want to take and book a ticket.

nancy sellers:
it can take me a minute to suzhou by train.

john smith:
i heard it is a very good job.

nancy sellers:
it's true, isn't it?

john smith:
yes, it's true.

nancy sellers:
ok. let's ask. first you should be
i just wanted to let you to know that we had decided to offer the position to someone else.we considered you very

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [26]:
save_loc = "/content/drive/MyDrive/Programming/AI_peter/output_files" #@param {type:"string"}


In [27]:
p_list = [["nancy powell:"+"\n", 
           "how are you doing?"+"\n", "\n", 
           "john smith:" + "\n"], 
           ["nancy powell:"+"\n", "why is "],
           ["can you help me with my homework?"+"\n", "\n", 
            "john smith:" + "\n"],
           ["john smith:" + "\n"],
           ["Hey I’m meeting the astrophysics professor via zoom after school any tips?"+"\n",
            "\n", "john smith:" + "\n"], 
]


prompts = ["".join(line) for line in p_list]

In [28]:
from datetime import datetime
import pprint as pp

ds_date_time = datetime.now().strftime("%m.%d.%Y")

base_header = "gpt-dailydialogue-textgen-{}".format(ds_date_time)
prompt_IDs = [base_header + "_file-{}.txt".format(i+1) for i in range(5, len(prompts)+11)]

prompt_mng = {}
for pid, text in zip(prompt_IDs, prompts):
    prompt_mng[pid] = text
pp.pprint(prompt_mng)

{'gpt-dailydialogue-textgen-10.22.2021_file-10.txt': 'Hey I’m meeting the '
                                                     'astrophysics professor '
                                                     'via zoom after school '
                                                     'any tips?\n'
                                                     '\n'
                                                     'john smith:\n',
 'gpt-dailydialogue-textgen-10.22.2021_file-6.txt': 'nancy powell:\n'
                                                    'how are you doing?\n'
                                                    '\n'
                                                    'john smith:\n',
 'gpt-dailydialogue-textgen-10.22.2021_file-7.txt': 'nancy powell:\nwhy is ',
 'gpt-dailydialogue-textgen-10.22.2021_file-8.txt': 'can you help me with my '
                                                    'homework?\n'
                                                    '\n'
                     

In [29]:
from os.path import join

for pfile, my_prompt in prompt_mng.items():
    ai.generate_to_file(
        n=50,
        batch_size=5,
        prompt=my_prompt,
        max_length=512,
        temperature=0.8,
        top_p=0.9,
        destination_path=join(save_loc, pfile)
    )


10/22/2021 06:49:32 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/AI_peter/output_files/gpt-dailydialogue-textgen-10.22.2021_file-6.txt


  0%|          | 0/50 [00:00<?, ?it/s]

10/22/2021 06:51:51 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/AI_peter/output_files/gpt-dailydialogue-textgen-10.22.2021_file-7.txt


  0%|          | 0/50 [00:00<?, ?it/s]

10/22/2021 06:54:11 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/AI_peter/output_files/gpt-dailydialogue-textgen-10.22.2021_file-8.txt


  0%|          | 0/50 [00:00<?, ?it/s]

10/22/2021 06:56:28 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/AI_peter/output_files/gpt-dailydialogue-textgen-10.22.2021_file-9.txt


  0%|          | 0/50 [00:00<?, ?it/s]

10/22/2021 06:58:48 — INFO — aitextgen — Generating 50 texts to /content/drive/MyDrive/Programming/AI_peter/output_files/gpt-dailydialogue-textgen-10.22.2021_file-10.txt


  0%|          | 0/50 [00:00<?, ?it/s]